In [ ]:
import open3d as o3d
from open3d import *
import numpy as np
import struct
from natsort import natsorted
import os
import time
from tqdm import tqdm 
import pickle  
import icp 
import copy
from numpy.linalg import inv 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
def convert_kitti_bin_to_pcd(binFilePath):
    # Load binary point cloud
    bin_pcd = np.fromfile(binFilePath, dtype=np.float32)

    # Reshape and drop reflection values
    points = bin_pcd.reshape((-1, 4))[:, 0:3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    # Convert to Open3D point cloud
    return pcd

input_dir = "00/"
input_files = natsorted(os.listdir(input_dir))

pcds = []

try : 
    fileObj = open('PCDs.pikle', 'rb')
    pcds = pickle.load(fileObj)
    fileObj.close()
except:
    pcds = []
    for in_file in tqdm(input_files):
        pcd = convert_kitti_bin_to_pcd(os.path.join(input_dir, in_file))
        pcd_np = np.asanyarray(pcd.points)
        pcds.append(pcd_np)
    
    fileObj = open('PCDs.pikle', 'wb')
    pickle.dump(pcds,fileObj)
    fileObj.close()

In [ ]:
def slam_icp(pcds,threshold=0.00001):
    transformation =[]
    world_map = []
    vis = o3d.visualization.Visualizer()
    vis.create_window()   
     
    for pc_index in tqdm(range(len(pcds))):
        point_cloud = pcds[pc_index]
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        pcd = pcd.voxel_down_sample(voxel_size=0.1)
        pcd_cp = copy.deepcopy(pcd)
        
        if pc_index == 0 :
            transformation.append(np.eye(4))
            world_map = np.array(pcd.points)
            world_map = np.round(np.array(pcd.points),decimals=1)
            world_map = np.unique(world_map,axis=0)
            continue 
        
        pcd_n1 = o3d.geometry.PointCloud()
        pcd_n1.points = o3d.utility.Vector3dVector(pcds[pc_index-1])
        pcd_n1 = pcd_n1.voxel_down_sample(voxel_size=0.1)
        pcd_n1_cp = copy.deepcopy(pcd_n1)    
        
        pcd_p = np.asarray(pcd.points)
        pcd_n1_p = np.asarray(pcd_n1.points)
        
        _min_count = min(pcd_p.shape[0],pcd_n1_p.shape[0])
        
        pcd_p = pcd_p[:_min_count]
        pcd_n1_p = pcd_n1_p[:_min_count]
            
        if pc_index == 1 :
            """reg_p2l = o3d.pipelines.registration.registration_icp(
                pcd,p_tilde[0], threshold, np.eye(4),
                o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200000))
            """
            T,_,__ = icp.icp(pcd_p,pcd_n1_p,max_iterations=100000,tolerance=0.00001)
            transformation.append(T)
            continue
        
        #reg_p2l = o3d.pipelines.registration.registration_icp(pcd_cp,pcd_n1, threshold, np.eye(4),o3d.pipelines.registration.TransformationEstimationPointToPoint(),o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200000))    
        T,_,__ = icp.icp(pcd_p,pcd_n1_p,max_iterations=100000,tolerance=0.00001)
        Tn = T
        transformation.append(Tn)
        
        for _ in range(len(transformation)-1,-1,-1):
            pcd_cp.transform(transformation[_])
            
        vis = o3d.visualization.Visualizer()
        vis.create_window(visible=False)
                
        world_map = np.concatenate([world_map,np.array(pcd_cp.points)])
        world_map = np.round(world_map,decimals=1) 
        world_map = np.unique(world_map,axis=0) 
        
        world_map_pcd = o3d.geometry.PointCloud()
        world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
        vis.add_geometry(world_map_pcd)
        vis.poll_events()
        vis.update_renderer()
        vis.capture_screen_image("./tracking.png")
        
    return world_map,transformation
    
    
slam_icp_result = slam_icp(pcds)


In [ ]:
def calculate_driving_lines(world_map,transformation,name="icp"):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    world_map_pcd = o3d.geometry.PointCloud()
    world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
    vis.add_geometry(world_map_pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image("./results/result_{0}_world_map_drive_path.png".format(name))
    vis.destroy_window()
        
    Centers = []
    Centers_gt = []
    gt_file = open("./gt_pose_00.txt").readlines()


    for pc_index in tqdm(range(len(transformation))):
        point_cloud = np.array([[0,0,100],[0,0,0]])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        pcd = pcd.voxel_down_sample(voxel_size=0.1)
        
        for _ in range(pc_index-1,-1,-1):
            pcd.transform(transformation[_])
            
        Centers.append(np.asarray(pcd.points)[0][:2])
        
        world_map = np.concatenate([world_map,np.array(pcd.points)])
            
        point_cloud = np.array([[0,0,0]])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        pcd = pcd.voxel_down_sample(voxel_size=0.1)
        
        gt = gt_file[pc_index]
        gt = gt.strip()
        gt = np.array(list(map(np.float64, gt.split()))).reshape(3,4)
        gt = np.concatenate([gt,np.array([[0,0,0,1]])])    
        pcd.transform(gt)
        Centers_gt.append(np.asarray(pcd.points)[0][:2])
        
        
        vis = o3d.visualization.Visualizer()
        vis.create_window(visible=False)
        
        world_map_pcd = o3d.geometry.PointCloud()
        world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
        vis.add_geometry(world_map_pcd)
        vis.poll_events()
        vis.update_renderer()
        vis.capture_screen_image("./results/result_{1}_world_map_{0}_drive_path.png".format(pc_index,name))
        vis.destroy_window()

    Centers = np.array(Centers)
    Centers = Centers/Centers.max(axis=0)
    plt.plot(Centers[:,1],Centers[:,0],label="Estimated driving line")

    Centers_gt = np.array(Centers_gt) * (-1)
    Centers_gt = Centers_gt/Centers_gt.max(axis=0)
    plt.plot(Centers_gt[:,0],Centers_gt[:,1],label="Ground truth driving line")
    plt.title("Driving lines")
    plt.ylabel("Y axis (Normalized & Aligned)")
    plt.xlabel("X axis (Normalized & Aligned)")

    plt.legend()
    plt.show()

world_map = slam_icp_result[0]
transformation = slam_icp_result[1] 
calculate_driving_lines(world_map,transformation,name="icp")

In [ ]:
fileObj = open('./slam_icp_result_transformation.pikle', 'rb')
tra = pickle.load(fileObj)
fileObj.close()